# Inspect action space
> 说明：本 notebook 用于生成 PySC2 动作空间（非 RAW / RAW）相关表格与示例，供 docs/action_space.md 使用。

> 若本地未安装或未配置 StarCraft II / pysc2，环境创建会失败，但动作函数表仍可生成。

In [ ]:
# 0) 基础：列出非 RAW 动作函数及其参数定义（不依赖 SC2 环境）
from pysc2.lib import actions
import json
import os

def repo_docs_dir():
    repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
    docs_dir = os.path.join(repo_root, 'docs')
    os.makedirs(docs_dir, exist_ok=True)
    return docs_dir

def arg_desc(arg):
    name = getattr(arg, 'name', '')
    sizes = getattr(arg, 'sizes', None)
    values = getattr(arg, 'values', None)
    values_names = None
    if values is not None:
        try:
            values_names = [getattr(v, 'name', str(v)) for v in list(values)]
        except Exception:
            values_names = None
    return {
        'name': name,
        'sizes': list(sizes) if sizes is not None else None,
        'values': values_names,
    }

non_raw = []
for fn in actions.FUNCTIONS:
    non_raw.append({
        'id': int(fn.id),
        'name': fn.name,
        'args': [a.name for a in fn.args],
        'arg_details': [arg_desc(a) for a in fn.args],
    })

# Print the JSON to notebook output instead of writing to a file
print(json.dumps(non_raw, ensure_ascii=False, indent=2))
print('Non-RAW FUNCTIONS count:', len(actions.FUNCTIONS))

Non-RAW FUNCTIONS count: 573
Wrote: c:\Users\ydx\OneDrive\桌面\实验室实习相关\第二阶段\rl\docs\generated_non_raw_actions.json


In [5]:
# 0.1) 动作参数类型定义（非 RAW / RAW）
from pysc2.lib import actions
import json
import os

def repo_docs_dir():
    repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
    docs_dir = os.path.join(repo_root, 'docs')
    os.makedirs(docs_dir, exist_ok=True)
    return docs_dir

def dump_arguments(arg_types, out_name):
    args = []
    for a in arg_types:
        name = getattr(a, 'name', '')
        sizes = getattr(a, 'sizes', None)
        values = getattr(a, 'values', None)
        values_names = None
        if values is not None:
            try:
                values_names = [getattr(v, 'name', str(v)) for v in list(values)]
            except Exception:
                values_names = None
        args.append({
            'name': name,
            'sizes': list(sizes) if sizes is not None else None,
            'values': values_names,
            'count': getattr(a, 'count', None),
        })
    out_path = os.path.join(repo_docs_dir(), out_name)
    with open(out_path, 'w', encoding='utf-8') as f:
        json.dump(args, f, ensure_ascii=False, indent=2)
    print('Wrote:', out_path, 'count:', len(args))

dump_arguments(actions.TYPES, 'generated_non_raw_arguments.json')
dump_arguments(actions.RAW_TYPES, 'generated_raw_arguments.json')

Wrote: c:\Users\ydx\OneDrive\桌面\实验室实习相关\第二阶段\rl\docs\generated_non_raw_arguments.json count: 13
Wrote: c:\Users\ydx\OneDrive\桌面\实验室实习相关\第二阶段\rl\docs\generated_raw_arguments.json count: 4


## 使用示例（非 RAW / RAW）

下面两段示例仅用于理解动作参数结构；真正执行时需要先创建 env 并从 obs 中取可用动作。

**非 RAW 示例**
```python
from pysc2.lib import actions
act = actions.FunctionCall(actions.FUNCTIONS.Move_screen.id, [[0], [x, y]])
`````

**RAW 示例**
```python
from pysc2.lib import actions
act = actions.FunctionCall(actions.RAW_FUNCTIONS.Move_pt.id, [[unit_tag], [x, y], [0]])
`````

In [6]:
# 1) RAW 动作函数表（不依赖 SC2 环境）
from pysc2.lib import actions
import json
import os

def repo_docs_dir():
    repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
    docs_dir = os.path.join(repo_root, 'docs')
    os.makedirs(docs_dir, exist_ok=True)
    return docs_dir

def arg_desc(arg):
    name = getattr(arg, 'name', '')
    sizes = getattr(arg, 'sizes', None)
    values = getattr(arg, 'values', None)
    values_names = None
    if values is not None:
        try:
            values_names = [getattr(v, 'name', str(v)) for v in list(values)]
        except Exception:
            values_names = None
    return {
        'name': name,
        'sizes': list(sizes) if sizes is not None else None,
        'values': values_names,
    }

raw = []
for fn in actions.RAW_FUNCTIONS:
    raw.append({
        'id': int(fn.id),
        'name': fn.name,
        'args': [a.name for a in fn.args],
        'arg_details': [arg_desc(a) for a in fn.args],
    })

out_path = os.path.join(repo_docs_dir(), 'generated_raw_actions.json')
with open(out_path, 'w', encoding='utf-8') as f:
    json.dump(raw, f, ensure_ascii=False, indent=2)

print('RAW_FUNCTIONS count:', len(actions.RAW_FUNCTIONS))
print('Wrote:', out_path)

RAW_FUNCTIONS count: 564
Wrote: c:\Users\ydx\OneDrive\桌面\实验室实习相关\第二阶段\rl\docs\generated_raw_actions.json


In [7]:
# 2) 非 RAW 环境：action_spec / available_actions / 参数范围示例
from absl import flags
import pysc2.run_configs
import json
import os
try:
    flags.FLAGS(['notebook', '--sc2_run_config=Windows'])
except Exception:
    pass
try:
    from pysc2.env import sc2_env
    from pysc2.lib import actions
    aif = sc2_env.AgentInterfaceFormat(
        feature_dimensions=sc2_env.Dimensions(screen=64, minimap=64),
        use_raw_actions=False,
    )
    env = sc2_env.SC2Env(
        map_name="MoveToBeacon",
        players=[sc2_env.Agent(sc2_env.Race.terran)],
        agent_interface_format=aif,
        step_mul=8,
        game_steps_per_episode=0,
        visualize=False,
    )
    try:
        spec = env.action_spec()
        timesteps = env.reset()
        obs = timesteps[0].observation
        avail = obs.get('available_actions', [])
        avail = [int(x) for x in list(avail)]
        avail_details = []
        for fid in avail:
            try:
                fn = actions.FUNCTIONS[fid]
                avail_details.append({'id': fid, 'name': fn.name, 'args': [a.name for a in fn.args]})
            except Exception:
                avail_details.append({'id': fid, 'name': '<unknown>', 'args': []})
        out = {
            'action_spec': str(spec),
            'available_actions': avail_details,
        }
        repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
        docs_dir = os.path.join(repo_root, 'docs')
        os.makedirs(docs_dir, exist_ok=True)
        out_path = os.path.join(docs_dir, 'generated_non_raw_available_actions.json')
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(out, f, ensure_ascii=False, indent=2)
        print('available_actions count:', len(avail_details))
        print('Wrote:', out_path)
    finally:
        try:
            env.close()
        except Exception:
            pass
except Exception as e:
    print('Non-RAW inspect failed:', e)
    import traceback; traceback.print_exc()

c:\Users\ydx\.conda\envs\PPO-minigame\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.10.19)
Hello from the pygame community. https://www.pygame.org/contribute.html
available_actions count: 6
Wrote: c:\Users\ydx\OneDrive\桌面\实验室实习相关\第二阶段\rl\docs\generated_non_raw_available_actions.json


In [8]:
# 3) RAW 环境：action_spec / 原始单位 tag / 参数范围示例
from absl import flags
import pysc2.run_configs
import json
import os
try:
    flags.FLAGS(['notebook', '--sc2_run_config=Windows'])
except Exception:
    pass
try:
    from pysc2.env import sc2_env
    from pysc2.lib import actions
    aif = sc2_env.AgentInterfaceFormat(
        feature_dimensions=sc2_env.Dimensions(screen=64, minimap=64),
        use_raw_actions=True,
        use_raw_units=True,
    )
    env = sc2_env.SC2Env(
        map_name="MoveToBeacon",
        players=[sc2_env.Agent(sc2_env.Race.terran)],
        agent_interface_format=aif,
        step_mul=8,
        game_steps_per_episode=0,
        visualize=False,
    )
    try:
        spec = env.action_spec()
        timesteps = env.reset()
        obs = timesteps[0].observation
        raw_units = obs.get('raw_units', [])
        sample_tags = []
        if len(raw_units) > 0:
            try:
                sample_tags = [int(u[30]) for u in raw_units[:10]]
            except Exception:
                sample_tags = []
        avail = obs.get('available_actions', [])
        avail_details = []
        if avail:
            for fid in list(avail):
                try:
                    fn = actions.RAW_FUNCTIONS[fid]
                    avail_details.append({'id': int(fid), 'name': fn.name, 'args': [a.name for a in fn.args]})
                except Exception:
                    avail_details.append({'id': int(fid), 'name': '<unknown>', 'args': []})
        out = {
            'action_spec': str(spec),
            'sample_unit_tags': sample_tags,
            'available_actions': avail_details,
        }
        repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
        docs_dir = os.path.join(repo_root, 'docs')
        os.makedirs(docs_dir, exist_ok=True)
        out_path = os.path.join(docs_dir, 'generated_raw_available_actions.json')
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(out, f, ensure_ascii=False, indent=2)
        print('raw_units count:', len(raw_units))
        print('Wrote:', out_path)
    finally:
        try:
            env.close()
        except Exception:
            pass
except Exception as e:
    print('RAW inspect failed:', e)
    import traceback; traceback.print_exc()

raw_units count: 3
Wrote: c:\Users\ydx\OneDrive\桌面\实验室实习相关\第二阶段\rl\docs\generated_raw_available_actions.json
